In [1]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
import random

import qiskit
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from enum import Enum

from dataclasses import dataclass

In [2]:
# Constants:
class Setting(Enum):
    PEEK = 1
    REVERSE_1 = 2
    REVERSE_2 = 3

# Size of the bipartite quantum system.
SYS_SIZE = 2

# Size of the systems held by the "friends" (Charlie and Debbie).
CHARLIE_SIZE = 3
DEBBIE_SIZE = 3

# Two output bits for Alice and Bob.
MEAS_SIZE = 2

In [11]:
# Quantum Register Layout:
# system; Charlie; Debbie; [Alice]; [Bob]

charlie_size=3
debbie_size=charlie_size

def bell(qc):
    qc.h(0)
    qc.cx(0, 1)
    return qc

def unitary_measure(qc, system_qubit, friend_qubit, size=1):
    for xx in range(0, size):
        qc.cx(system_qubit, friend_qubit+xx)
        
def ewfs(qc, alice, bob, angles=None):
    if angles == None:
        angles = [np.pi/2, np.pi/2]
    
    bell(qc)
    
    # Charlie measurement
    unitary_measure(qc, 0, 2, size=charlie_size)
    # Debbie measurement
    unitary_measure(qc, 1, 2+charlie_size, size=debbie_size)
    
    # 0: Charlie system; 2: Charlie; [0]: Alice
    if alice == 1:
        # ask Charlie for the outcome
        # we pick a random qubit from Charlie's register
        random_offset = random.randint(0, charlie_size-1)
        qc.measure(2+random_offset, 0)
    else:
        # undo the measurement
        unitary_measure(qc, 0, 2, size=charlie_size)
        # Alice measures directly in a different basis
        qc.rz(np.pi/2, 0)
        qc.rx(np.pi/2, 0)
        if alice == 2:
            qc.rz(angles[0], 0)
        elif alice == 3:
            qc.rz(angles[1], 0)
        qc.measure(0, 0)
    
    # 1: Debbie system; 3: Debbie; [1]: Bob
    if bob == 1:
        # ask Debbie for the outcome
        # we pick a random qubit from Debbie's register
        random_offset = random.randint(0, debbie_size-1)
        qc.measure(2+charlie_size+random_offset, 1)
    else:
        # undo the measurement
        unitary_measure(qc, 1, 2+charlie_size, size=debbie_size)
        # Bob measures directly in a different basis
        qc.rz(np.pi/2, 0)
        qc.rx(np.pi/2, 1)
        if bob == 2:
            qc.rz(angles[0], 1)
        elif bob == 3:
            qc.rz(angles[1], 1)
        qc.measure(1, 1)

    return qc
#qc = make_qc()
qc = QuantumCircuit(sum([SYS_SIZE, CHARLIE_SIZE, DEBBIE_SIZE]), MEAS_SIZE) 

ewfs(qc, 1, 1)
qc.draw()

┌───┐                                      
q_0: ┤ H ├──■────■─────────■─────────■──────────
     └───┘┌─┴─┐  │         │         │          
q_1: ─────┤ X ├──┼────■────┼────■────┼────■─────
          └───┘┌─┴─┐  │    │    │    │    │     
q_2: ──────────┤ X ├──┼────┼────┼────┼────┼─────
               └───┘  │  ┌─┴─┐  │    │    │     
q_3: ─────────────────┼──┤ X ├──┼────┼────┼─────
                      │  └───┘  │  ┌─┴─┐  │  ┌─┐
q_4: ─────────────────┼─────────┼──┤ X ├──┼──┤M├
                    ┌─┴─┐ ┌─┐   │  └───┘  │  └╥┘
q_5: ───────────────┤ X ├─┤M├───┼─────────┼───╫─
                    └───┘ └╥┘ ┌─┴─┐       │   ║ 
q_6: ──────────────────────╫──┤ X ├───────┼───╫─
                           ║  └───┘     ┌─┴─┐ ║ 
q_7: ──────────────────────╫────────────┤ X ├─╫─
                           ║            └───┘ ║ 
c: 2/══════════════════════╩══════════════════╩═
                           1                  0

In [3]:
def bell(qc):
    qc.h(0)
    qc.cx(0, 1)
    return qc

In [4]:
def unitary_measure(qc, system_qubit, friend_qubit, size=1):
    for xx in range(0, size):
        qc.cx(system_qubit, friend_qubit+xx)

In [5]:
alice = Setting.PEEK
bob = Setting.PEEK

# Circuit: system + Charlie + Debbie, [Alice+Bob]
qc = QuantumCircuit(sum([SYS_SIZE, CHARLIE_SIZE, DEBBIE_SIZE]), MEAS_SIZE) 

bell(qc)

# Charlie measurement
unitary_measure(qc, 0, 2, size=CHARLIE_SIZE)
# Debbie measurement
unitary_measure(qc, 1, 2+CHARLIE_SIZE, size=DEBBIE_SIZE)

# 0: Charlie system; 2: Charlie; [0]: Alice
if alice == Setting.PEEK:
    # ask Charlie for the outcome
    # we pick a random qubit from Charlie's register
    random_offset = random.randint(0, CHARLIE_SIZE-1)
    qc.measure(2 + random_offset, 0)
else:
    # undo the measurement
    unitary_measure(qc, 0, 2, size=CHARLIE_SIZE)
    # Alice measures directly in a different basis
    qc.rz(np.pi/2, 0)
    qc.rx(np.pi/2, 0)
    if alice == Setting.REVERSE_1:
        qc.rz(angles[0], 0)
    elif alice == Setting.REVERSE_2:
        qc.rz(angles[1], 0)
    qc.measure(0, 0)

    
# 1: Debbie system; 3: Debbie; [1]: Bob
if bob == Setting.PEEK:
    # ask Debbie for the outcome
    # we pick a random qubit from Debbie's register
    random_offset = random.randint(0, DEBBIE_SIZE-1)
    qc.measure(2+CHARLIE_SIZE+random_offset, 1)
else:
    # undo the measurement
    unitary_measure(qc, 1, 2+charlie_size, size=DEBBIE_SIZE)
    # Bob measures directly in a different basis
    qc.rz(np.pi/2, 0)
    qc.rx(np.pi/2, 1)
    if bob == Setting.REVERSE_1:
        qc.rz(angles[0], 1)
    elif bob == Setting.REVERSE_2:
        qc.rz(angles[1], 1)
    qc.measure(1, 1)
    
print(qc)

     ┌───┐                                      
q_0: ┤ H ├──■────■─────────■─────────■──────────
     └───┘┌─┴─┐  │         │         │          
q_1: ─────┤ X ├──┼────■────┼────■────┼────■─────
          └───┘┌─┴─┐  │    │    │    │    │     
q_2: ──────────┤ X ├──┼────┼────┼────┼────┼─────
               └───┘  │  ┌─┴─┐  │    │    │  ┌─┐
q_3: ─────────────────┼──┤ X ├──┼────┼────┼──┤M├
                      │  └───┘  │  ┌─┴─┐  │  └╥┘
q_4: ─────────────────┼─────────┼──┤ X ├──┼───╫─
                    ┌─┴─┐ ┌─┐   │  └───┘  │   ║ 
q_5: ───────────────┤ X ├─┤M├───┼─────────┼───╫─
                    └───┘ └╥┘ ┌─┴─┐       │   ║ 
q_6: ──────────────────────╫──┤ X ├───────┼───╫─
                           ║  └───┘     ┌─┴─┐ ║ 
q_7: ──────────────────────╫────────────┤ X ├─╫─
                           ║            └───┘ ║ 
c: 2/══════════════════════╩══════════════════╩═
                           1                  0 


In [6]:
def prepare_bipartite_system():
    """Generates the state: 1/sqrt(3) * (|00> + |01> + |10>)"""
    qc = QuantumCircuit(2)
    qc.ry(2 * np.arccos(np.sqrt(2/3)), 0)
    qc.ry(np.pi/4, 1)
    qc.x(0)
    qc.cx(0, 1)
    qc.x(0)
    qc.ry(-np.pi/4, 1) 
    return qc

In [ ]:
def unitary_measure(qc, system_qubit, friend_qubit, size=1):
    for xx in range(size):
        qc.cx(system_qubit, friend_qubit + xx)

In [ ]:
def cnot_ladder(system_qubit, friend_qubit, friend_size):
    circuit = QuantumCircuit(2* SYSTEM_SIZE + friend_size)
    for i in range(friend_size):
        circuit.cx(system_qubit, friend_qubit + i)
    return circuit

In [ ]:
alice, bob = Setting.PEEK, Setting.PEEK

# Circuit: system + Charlie + Debbie, [Alice+Bob]
qc = QuantumCircuit(sum([SYS_SIZE, CHARLIE_SIZE, DEBBIE_SIZE]), MEAS_SIZE) 

# State preparation:
qc = qc.compose(prepare_bipartite_system())

#qc = qc.compose(cnot_ladder(system_qubit=0, friend_qubit=system_size, friend_size=charlie_size))
#qc = qc.compose(cnot_ladder(system_qubit=1, friend_qubit=system_size, friend_size=debbie_size))

unitary_measure(qc, 0, SYS_SIZE, size=CHARLIE_SIZE)
unitary_measure(qc, 1, SYS_SIZE + CHARLIE_SIZE, size=DEBBIE_SIZE)

# # 0: Charlie system; 2: Charlie; [0]: Alice
# if alice is Setting.PEEK:
#     # Ask Charlie for the outcome. Pick a random qubit from Charlie's register.
#     random_offset = random.randint(0, CHARLIE_SIZE-1)
#     qc.measure(SYS_SIZE + random_offset, 0)

# elif alice in [Setting.REVERSE_1, Setting.REVERSE_2]:
#     # undo the measurement
#     unitary_measure(qc, 0, 2, size=CHARLIE_SIZE)
#     # Alice measures directly in a different basis
#     qc.rz(np.pi/2, 0)
#     qc.rx(np.pi/2, 0)
#     if alice is Setting.REVERSE_1:
#         qc.rz(angles[0], 0)
#     elif alice == Setting.REVERSE_2:
#         qc.rz(angles[1], 0)
#     qc.measure(0, 0)

print(qc)

In [ ]:
def bell(qc):
    qc.h(0)
    qc.cx(0, 1)
    return qc

In [ ]:
svsim = Aer.get_backend('statevector_simulator')
result = svsim.run(circuit).result().get_counts()

In [ ]:
def unitary_measure(qc, system_qubit, friend_qubit, size=1):
    for xx in range(0, size):
        qc.cx(system_qubit, friend_qubit+xx)

In [ ]:
def ewfs(qc, alice, bob, angles=None):
    if angles == None:
        angles = [np.pi/2, np.pi/2]
    
    bell(qc)
    
    # Charlie measurement
    unitary_measure(qc, 0, 2, size=charlie_size)
    
    # Debbie measurement
    unitary_measure(qc, 1, 2+charlie_size, size=debbie_size)
    
    # 0: Charlie system; 2: Charlie; [0]: Alice
    if alice == 1:
        # ask Charlie for the outcome
        # we pick a random qubit from Charlie's register
        random_offset = random.randint(0, charlie_size-1)
        qc.measure(2+random_offset, 0)
    else:
        # undo the measurement
        unitary_measure(qc, 0, 2, size=charlie_size)
        # Alice measures directly in a different basis
        qc.rz(np.pi/2, 0)
        qc.rx(np.pi/2, 0)
        if alice == 2:
            qc.rz(angles[0], 0)
        elif alice == 3:
            qc.rz(angles[1], 0)
        qc.measure(0, 0)
    
    # 1: Debbie system; 3: Debbie; [1]: Bob
    if bob == 1:
        # ask Debbie for the outcome
        # we pick a random qubit from Debbie's register
        random_offset = random.randint(0, debbie_size-1)
        qc.measure(2+charlie_size+random_offset, 1)
    else:
        # undo the measurement
        unitary_measure(qc, 1, 2+charlie_size, size=debbie_size)
        # Bob measures directly in a different basis
        qc.rz(np.pi/2, 0)
        qc.rx(np.pi/2, 1)
        if bob == 2:
            qc.rz(angles[0], 1)
        elif bob == 3:
            qc.rz(angles[1], 1)
        qc.measure(1, 1)

    return qc

In [ ]:
qc = make_qc()
ewfs(qc, 1, 2)
qc.draw()

In [ ]:
qc = make_qc()
print(qc)

In [ ]:
qc = make_qc()

bell(qc)

# Charlie measurement
unitary_measure(qc, 0, 2, size=charlie_size)

# Debbie measurement
unitary_measure(qc, 1, 2+charlie_size, size=debbie_size)

print(qc)